In [1]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

from keras.models import load_model

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


In [0]:
tf.enable_eager_execution()

In [0]:
BATCH_SIZE = 128 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.01 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 20 #@param {type:"integer"}

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out, v_stride):
    super().__init__()
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, strides=v_stride, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)

  def call(self, inputs):
    return (tf.nn.relu(self.bn(self.conv(inputs))))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self,c_in, c_out, stride = 1):
    super().__init__()
    self.stride = stride
    self.c_in = c_in
    self.c_out = c_out
    self.convbn1 = ConvBN(c_out,v_stride = 1)
    self.conv1 = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3,strides=1,padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn1 = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.adjconv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=1,strides=stride,padding="SAME", kernel_initializer=init_pytorch, use_bias=False)

    self.convbn2 = ConvBN(c_out, v_stride = 1)
    self.conv2 = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3,strides=1,padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn2 = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)


  def call(self, inputs):
    h1 = self.convbn1(inputs)
    h1 = self.bn1(self.conv1(h1))

    if (self.c_in != self.c_out):
      h1 = self.adjconv(inputs) + h1
    else:
      h1 = inputs + h1
      
    h1 = tf.nn.relu(h1)  


    h2 = self.convbn2(h1)
    h2 = self.bn2(self.conv2(h2))

    h = h1 + h2
      
    h = tf.nn.relu(h)  

    return h

    

In [0]:
class ResNet_V2_18(tf.keras.Model):
  def __init__(self, c=64):
    super().__init__()
    self.convbn = ConvBN(c, v_stride=1)
    self.blk1 = ResBlk(c, c)
    self.blk2 = ResBlk(c, c*2)
    self.blk3 = ResBlk(c*2, c*4)
    self.blk4 = ResBlk(c*4, c*8)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
  
  def call(self, x, y):
    
    h1 = self.blk4(self.blk3(self.blk2(self.blk1(self.convbn(x)))))
    h = self.pool(h1)
    h = self.linear(h)
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct


In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

train_mean = np.array([0.4914, 0.4822, 0.4465])
train_std = np.array([0.2023, 0.1994, 0.2010])

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

In [0]:
model = ResNet_V2_18()
batches_per_epoch = len_train//BATCH_SIZE + 1

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [11]:

t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)
prevBestAcc = 0.0

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
  
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)
  modelPathName = '/content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5'

  currentTestAcc = (test_acc/len_test)
  
  if(currentTestAcc > prevBestAcc):
    print('Accuracy improved from {} to {}. Saving model {}'.format(prevBestAcc,currentTestAcc,modelPathName))
    prevBestAcc = currentTestAcc
    model.save_weights(modelPathName)

  if(currentTestAcc >= 0.92):
    print('Accuracy of >= 92% achieved. Exiting model training.') 
    model.save_weights(modelPathName)
    break 
    
  


epoch: 1 lr: 0.0025 train loss: 1.8360749334716797 train acc: 0.37942 val loss: 1.362384853744507 val acc: 0.5281 time: 263.24363565444946
Accuracy improved from 0.0 to 0.5281. Saving model /content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5



epoch: 2 lr: 0.005 train loss: 1.2187800074768067 train acc: 0.57236 val loss: 1.1410848329544068 val acc: 0.5989 time: 521.5126769542694
Accuracy improved from 0.5281 to 0.5989. Saving model /content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5



epoch: 3 lr: 0.0075 train loss: 0.9475689175415039 train acc: 0.67432 val loss: 0.9083898184776306 val acc: 0.6881 time: 780.181676864624
Accuracy improved from 0.5989 to 0.6881. Saving model /content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5



epoch: 4 lr: 0.01 train loss: 0.7877906192779541 train acc: 0.72736 val loss: 1.1036533176422119 val acc: 0.6627 time: 1037.6440482139587



epoch: 5 lr: 0.009375 train loss: 0.6603953889465332 train acc: 0.77194 val loss: 0.6421489703178406 val acc: 0.7831 time: 1295.5036289691925
Accuracy improved from 0.6881 to 0.7831. Saving model /content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5


...


epoch: 16 lr: 0.0025000000000000005 train loss: 0.0999302628660202 train acc: 0.96654 val loss: 0.304956880569458 val acc: 0.9035 time: 4133.791127920151



epoch: 17 lr: 0.001875 train loss: 0.07756929563045502 train acc: 0.97542 val loss: 0.3150089140892029 val acc: 0.905 time: 4390.972702264786
Accuracy improved from 0.9047 to 0.905. Saving model /content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5



epoch: 18 lr: 0.0012499999999999994 train loss: 0.060788690927028655 train acc: 0.9815 val loss: 0.3171021216392517 val acc: 0.9065 time: 4648.621041297913
Accuracy improved from 0.905 to 0.9065. Saving model /content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5



epoch: 19 lr: 0.0006250000000000006 train loss: 0.04511489334106445 train acc: 0.98704 val loss: 0.30522286529541015 val acc: 0.9126 time: 4905.908576726913
Accuracy improved from 0.9065 to 0.9126. Saving model /content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5



epoch: 20 lr: 0.0 train loss: 0.036473881087303164 train acc: 0.99082 val loss: 0.2989235047340393 val acc: 0.9133 time: 5163.271771907806
Accuracy improved from 0.9126 to 0.9133. Saving model /content/drive/My Drive/EVA_Model/Assignment13_BestModelWeights.h5
